In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.metrics import  roc_auc_score

import pickle 
import logging


# from config.config import SQLQuery
# querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [2]:
from preprocess import Convert,MissingValues,Outlier,FeatureSelection
from feature_transformation import Scaler,Transform,Selection
from model_building import split_test_train, feature_encoding, classification_models
from model_evaluations import model_metrics, feature_importance, probability_bins, cross_validation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

from statsmodels.stats.outliers_influence import variance_inflation_factor

# object initiation 
tf = Transform()
sel = Selection()
ft = FeatureSelection()
cv = Convert()
mv = MissingValues()
ot = Outlier()

# set seed
seed = 37

In [3]:
df_raw = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/final_dataset_2.pkl')
df_raw.shape

(646, 255)

In [4]:
feat_list = ['credit_size_3M',
 'pos_size_1M',
 'ratio_amt_credits_less_100_credits_grt_500_3M',
 'median_credit_amt_1M',
 'ratio_pos_size_1M_6M',
 'median_pos_amt_3M',
 'count_credits_grt_500_3M',
 'loans_size_6M',
 'ach_count_6M']

In [5]:
keep_cols = ['business_id', 'lending_business_id', 'decision_date', 'target', 'txn_flag']

In [6]:
df_raw = df_raw[keep_cols + feat_list]
df_raw.shape

(646, 14)

In [7]:
df_raw.sample(5)

,business_id,lending_business_id,decision_date,target,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
140,69b1a76e-be39-44f5-bcde-fb710b6f1556,a97237da-ec1d-47c5-a1d0-31a485f903aa,2023-03-09,1,1.0,847.708333,NaN,0.012944,NaN,NaN,NaN,2.0,NaN,NaN
150,32b8f9b3-6bdf-47ae-8e72-08d4de48fa64,b33afb10-8c94-4fc2-8843-44ad7a05ef28,2023-02-08,0,1.0,1017.580000,1.375,0.003940,1300.0,0.578947,2.75,6.0,NaN,NaN
518,7997c204-c4eb-4ad6-9601-70badf0f07b6,d6eac83f-7df8-4f0c-a70d-14218a70d117,2023-01-28,0,1.0,447.857143,NaN,NaN,267.5,NaN,NaN,3.0,NaN,NaN
169,68c41211-5006-4061-84e4-6dc0927d6c50,f3da1132-e72f-49fd-bc99-d92f514bb902,2023-02-15,0,1.0,360.911667,NaN,0.002532,275.0,NaN,NaN,8.0,NaN,NaN
468,90009ab3-dc99-41c8-8315-19c066ca3c89,80534a0e-bec1-42e6-a247-6b7f8806a80e,2023-02-01,0,1.0,1299.341515,NaN,0.010341,675.0,NaN,NaN,13.0,NaN,NaN


In [8]:
df_raw.describe()

,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
count,646.000000,620.000000,256.000000,4.870000e+02,573.000000,256.000000,363.000000,579.000000,129.000000,203.000000
mean,0.986068,1900.288971,1003.794158,5.192965e-02,1189.955541,0.743371,1188.203140,14.096718,3947.993256,12.753695
std,0.117299,4530.977238,3305.248697,1.634456e-01,3382.613029,0.576655,3383.187711,15.195985,7238.932550,25.483802
min,0.000000,0.155000,0.150000,7.307604e-08,0.010000,0.001247,0.010000,1.000000,0.290000,1.000000
25%,1.000000,390.699894,89.477500,2.118917e-03,118.860000,0.331781,100.000000,5.000000,150.000000,2.000000
50%,1.000000,774.588458,281.750000,9.341087e-03,369.600000,0.692535,300.000000,10.000000,783.333333,5.000000
75%,1.000000,1663.454220,920.205937,3.279657e-02,1000.000000,0.998628,882.130000,17.000000,4423.255556,14.500000
max,1.000000,59653.400000,46250.000000,1.794795e+00,51080.000000,4.346746,40480.000000,137.000000,50670.975000,263.000000


In [9]:
median_dict = {'credit_size_3M' : 774.588458,
                'pos_size_1M' : 281.750000,
                'ratio_amt_credits_less_100_credits_grt_500_3M' : 9.341087e-03,
                'median_credit_amt_1M' : 369.600000,
                'ratio_pos_size_1M_6M' : 0.692535,
                'median_pos_amt_3M' : 300,
                'count_credits_grt_500_3M' : 10,
                'loans_size_6M' : 783.3,
                'ach_count_6M' : 5}

In [10]:
df_raw[df_raw['txn_flag'] == 0] = df_raw[df_raw['txn_flag'] == 0].fillna(median_dict)

In [11]:
df_raw = df_raw.fillna(0)
df_raw.isnull().sum().sum()

0

In [12]:
df_raw.drop('txn_flag',axis=1,inplace=True)

In [13]:
df_raw['target'] = df_raw['target'].astype('int')

In [14]:
feat_list

['credit_size_3M',
 'pos_size_1M',
 'ratio_amt_credits_less_100_credits_grt_500_3M',
 'median_credit_amt_1M',
 'ratio_pos_size_1M_6M',
 'median_pos_amt_3M',
 'count_credits_grt_500_3M',
 'loans_size_6M',
 'ach_count_6M']

In [15]:
transformed_vars = df_raw.copy()

In [16]:
# 1
# transform
col         = 'credit_size_3M'
conditions  = [ transformed_vars[col] < 178.8,
                (transformed_vars[col] >= 178.9 ) & (transformed_vars[col] < 392.56),
                (transformed_vars[col] >= 392.56 ) & (transformed_vars[col] < 2274.8),
                transformed_vars[col] >= 2274.8 ]

choices     = [-1.056997,-0.484478,0.184337,1.294378]
    
transformed_vars["credit_size_3M"] = np.select(conditions, choices, default=np.nan)


# 2
# transform
col         = 'pos_size_1M'
conditions  = [ transformed_vars[col] < 235.11, 
                (transformed_vars[col] >= 235.11 ) & (transformed_vars[col] < 979.44), 
                transformed_vars[col] >= 979.44 ]

choices     = [-0.152312,0.159398,1.699843]
    
transformed_vars["pos_size_1M"] = np.select(conditions, choices, default=np.nan)

# 3
# transform
col         = 'ratio_amt_credits_less_100_credits_grt_500_3M'
conditions  = [ transformed_vars[col] <= 0,
                transformed_vars[col] > 0 ]

choices     = [-0.112536,0.045514]
    
transformed_vars["ratio_amt_credits_less_100_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

# 4
# transform
col         = 'median_credit_amt_1M'
conditions  = [ transformed_vars[col] < 102,
                (transformed_vars[col] >= 102 ) & (transformed_vars[col] < 475.2),
                transformed_vars[col] >= 475.2 ]

choices     = [-0.444738,-0.119068,0.550264]
    
transformed_vars["median_credit_amt_1M"] = np.select(conditions, choices, default=np.nan)


# 5
# transform
col         = 'ratio_pos_size_1M_6M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 1.21),
                transformed_vars[col] >= 1.21 ]

choices     = [-0.164722,0.141314,1.140227]
    
transformed_vars["ratio_pos_size_1M_6M"] = np.select(conditions, choices, default=np.nan)

# 6
# transform
col         = 'median_pos_amt_3M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 486.25),
                transformed_vars[col] >= 486.25 ]

choices     = [-0.25334,-0.127284,0.968955]
    
transformed_vars["median_pos_amt_3M"] = np.select(conditions, choices, default=np.nan)

# 7
# transform
col         = 'count_credits_grt_500_3M'
conditions  = [ transformed_vars[col] < 1.5,
                (transformed_vars[col] >= 1.5 ) & (transformed_vars[col] < 15.5),
                transformed_vars[col] >= 15.5 ]

choices     = [-0.450862,-0.000945,0.284561]
    
transformed_vars["count_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

# 8
# transform
col         = 'loans_size_6M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 3317),
                transformed_vars[col] >= 3317 ]

choices     = [0.08126,-0.605012,0.814805]
    
transformed_vars["loans_size_6M"] = np.select(conditions, choices, default=np.nan)


# 9
# transform
col         = 'ach_count_6M'
conditions  = [ transformed_vars[col] < 6.5,
                transformed_vars[col] >= 6.5 ]

choices     = [-0.022101,0.149246]
    
transformed_vars["ach_count_6M"] = np.select(conditions, choices, default=np.nan)


In [17]:
transformed_vars.head()

,business_id,lending_business_id,decision_date,target,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,0,0.184337,-0.152312,0.045514,-0.119068,-0.164722,-0.127284,-0.000945,0.081260,-0.022101
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,0,0.184337,-0.152312,0.045514,0.550264,-0.164722,-0.127284,0.284561,-0.605012,-0.022101
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,0,1.294378,-0.152312,-0.112536,-0.444738,-0.164722,-0.253340,-0.450862,0.081260,-0.022101
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,0,1.294378,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,0,0.184337,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101


In [18]:
logreg_model = pickle.load(open('/Users/shashankgupta/Documents/code/git_project/plaid_credit/models/final_model.pkl', 'rb'))

In [19]:
# Scoring on full-data

scored_df = transformed_vars.copy()
scored_df['pred_proba'] = logreg_model.predict_proba(scored_df[feat_list])[:,1]

scored_df['odds'] = scored_df['pred_proba'] / (1-scored_df['pred_proba'])
scored_df['log_odds'] = np.log(scored_df['odds'])
scored_df['Ext_IN1_score'] = 601.7 - (86.6 * scored_df['log_odds'])
scored_df.Ext_IN1_score.describe()

count    646.000000
mean     618.060330
std       76.272747
min      469.821422
25%      567.695876
50%      601.579053
75%      655.069967
max      887.743320
Name: Ext_IN1_score, dtype: float64

In [20]:
scored_df.head()

,business_id,lending_business_id,decision_date,target,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M,pred_proba,odds,log_odds,Ext_IN1_score
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,0,0.184337,-0.152312,0.045514,-0.119068,-0.164722,-0.127284,-0.000945,0.081260,-0.022101,0.521805,1.091197,0.087275,594.141987
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,0,0.184337,-0.152312,0.045514,0.550264,-0.164722,-0.127284,0.284561,-0.605012,-0.022101,0.596929,1.480956,0.392687,567.693263
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,0,1.294378,-0.152312,-0.112536,-0.444738,-0.164722,-0.253340,-0.450862,0.081260,-0.022101,0.231759,0.301675,-1.198407,705.482011
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,0,1.294378,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.240053,0.315882,-1.152387,701.496684
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,0,0.184337,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.439750,0.784917,-0.242177,622.672553


In [21]:
def bin_logic(x):
    if x < 0.261540:
        return 5
    if x < 0.455167:
        return 4
    if x < 0.530817:
        return 3
    if x < 0.628923:
        return 2
    return 1

In [22]:
scored_df['Ext_IN1_bin'] = scored_df['pred_proba'].apply(lambda x: bin_logic(x))

In [23]:
scored_df.head()

,business_id,lending_business_id,decision_date,target,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M,pred_proba,odds,log_odds,Ext_IN1_score,Ext_IN1_bin
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,0,0.184337,-0.152312,0.045514,-0.119068,-0.164722,-0.127284,-0.000945,0.081260,-0.022101,0.521805,1.091197,0.087275,594.141987,3
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,0,0.184337,-0.152312,0.045514,0.550264,-0.164722,-0.127284,0.284561,-0.605012,-0.022101,0.596929,1.480956,0.392687,567.693263,2
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,0,1.294378,-0.152312,-0.112536,-0.444738,-0.164722,-0.253340,-0.450862,0.081260,-0.022101,0.231759,0.301675,-1.198407,705.482011,5
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,0,1.294378,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.240053,0.315882,-1.152387,701.496684,5
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,0,0.184337,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.439750,0.784917,-0.242177,622.672553,4


In [25]:
scored_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 646 entries, 0 to 645
Data columns (total 18 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   business_id                                    646 non-null    object 
 1   lending_business_id                            646 non-null    object 
 2   decision_date                                  646 non-null    object 
 3   target                                         646 non-null    int64  
 4   credit_size_3M                                 646 non-null    float64
 5   pos_size_1M                                    646 non-null    float64
 6   ratio_amt_credits_less_100_credits_grt_500_3M  646 non-null    float64
 7   median_credit_amt_1M                           646 non-null    float64
 8   ratio_pos_size_1M_6M                           646 non-null    float64
 9   median_pos_amt_3M                              646 non-null

In [29]:
t = scored_df.groupby('Ext_IN1_bin',as_index=False)['target'].agg(['sum','count']).rename(columns={'sum':'count_of_bads','count':'total_accs'})
t['bad_rate'] = np.round(t['count_of_bads']/t['total_accs'],2)
t

,Ext_IN1_bin,count_of_bads,total_accs,bad_rate
0,1,55,139,0.40
1,2,28,116,0.24
2,3,29,149,0.19
3,4,15,124,0.12
4,5,6,118,0.05


In [38]:
t1 = scored_df.groupby('Ext_IN1_bin', as_index=False)['Ext_IN1_score'].min().rename(columns={'Ext_IN1_score':'min_score'})
t2 = scored_df.groupby('Ext_IN1_bin', as_index=False)['Ext_IN1_score'].max().rename(columns={'Ext_IN1_score':'max_score'})

t3 = scored_df.groupby('Ext_IN1_bin', as_index=False)['pred_proba'].min().rename(columns={'pred_proba':'min_prob'})
t4 = scored_df.groupby('Ext_IN1_bin', as_index=False)['pred_proba'].max().rename(columns={'pred_proba':'max_prob'})

res = t1.merge(t2, on='Ext_IN1_bin')
res = res.merge(t3, on='Ext_IN1_bin')
res = res.merge(t4, on='Ext_IN1_bin')
res.to_clipboard()

In [35]:
scored_df.groupby('Ext_IN1_bin', as_index=False)['Ext_IN1_score'].max()

,Ext_IN1_bin,Ext_IN1_score
0,1,469.821422
1,2,557.045827
2,3,591.011509
3,4,619.916659
4,5,691.589243


In [30]:
logreg_model.feature_names_in_

array(['credit_size_3M', 'pos_size_1M',
       'ratio_amt_credits_less_100_credits_grt_500_3M',
       'median_credit_amt_1M', 'ratio_pos_size_1M_6M',
       'median_pos_amt_3M', 'count_credits_grt_500_3M', 'loans_size_6M',
       'ach_count_6M'], dtype=object)

In [32]:
logreg_model.coef_

array([[-0.81997815, -0.85685935,  0.72385091, -0.34517684, -0.35318817,
        -0.1268506 ,  0.86565206, -0.42155525,  0.18572408]])

In [33]:
logreg_model.intercept_

array([-0.00127315])

In [ ]:
# scored_df.to_csv('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/scored_dataset.csv')

In [ ]:
# scored_df.to_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/scored_dataset.pkl')